In [1]:
from river import anomaly
from river.anomaly import OneClassSVM
from river import feature_extraction as fx

from skmultiflow.data import FileStream             # create stream from file
from sklearn.model_selection import ParameterGrid   # hyperparameter combinations

from prequential import run_prequential

In [2]:
data_loader = FileStream(filepath='../merged.csv')

In [3]:
ref_sample, _ = data_loader.next_sample(50)
data_loader.reset()

In [4]:
parameters = {
    'q': [0.4, 0.5, 0.6, 0.7, 0.8],
    'gamma': [0.0001, 0.001, 0.01, 0.1, 1],
    'nu': [0.02, 0.05, 0.1, 0.5]
}

svm_grid = ParameterGrid(parameters)

In [5]:
best_params = None
best_score = 0
best_metrics = []

results = []

for params in svm_grid:
    print(f"Evaluating parameters: {params}")
    data_loader.restart()

    svm_model = (
        fx.RBFSampler(gamma=params['gamma']) | OneClassSVM(nu=params['nu'])
    )

    svm_classifier = anomaly.QuantileFilter(
        svm_model,
        q=params['q']
    )

    # tracemalloc.start()

    accuracy, precision, recall, f1, avg_processing_time = run_prequential(
        classifier=svm_classifier,
        stream=data_loader,
        feature_selector = None,
        preq_samples=100000
    )

    # current_mem, peak_mem = tracemalloc.get_traced_memory()
    # tracemalloc.stop()

    print(f"Accuracy: {accuracy:.6f}, Precision: {precision:.6f}, Recall: {recall:.6f}, F1: {f1:.6f}")
    print(f"Average processing time: {avg_processing_time}")
    # print(f"Current memory usage (KB): {current_mem / 1024}")
    # print(f"Peak memory usage (KB): {peak_mem / 1024}")

    avg_score = (accuracy + precision + recall + f1) / 4

    if avg_score > best_score:
        best_params = params
        best_score = avg_score
        best_metrics = [accuracy, precision, recall, f1]
    
    results.append({
        **params,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'avg_score': avg_score,
        'avg_processing_time': avg_processing_time
    })

print("Grid Search Completed.")
print(f"Best Parameters: {best_params}")
print(f"Best Metrics: {best_metrics}")

Evaluating parameters: {'gamma': 0.0001, 'nu': 0.02, 'q': 0.4}
Accuracy: 0.593060, Precision: 0.562872, Recall: 0.833140, F1: 0.671844
Average processing time: 0.006374400489000011
Evaluating parameters: {'gamma': 0.0001, 'nu': 0.02, 'q': 0.5}
Accuracy: 0.432280, Precision: 0.445843, Recall: 0.557500, F1: 0.495459
Average processing time: 0.007194748582999424
Evaluating parameters: {'gamma': 0.0001, 'nu': 0.02, 'q': 0.6}
Accuracy: 0.484080, Precision: 0.485830, Recall: 0.545820, F1: 0.514081
Average processing time: 0.007345416558000258
Evaluating parameters: {'gamma': 0.0001, 'nu': 0.02, 'q': 0.7}
Accuracy: 0.433740, Precision: 0.438795, Recall: 0.475040, F1: 0.456199
Average processing time: 0.0073412518710002815
Evaluating parameters: {'gamma': 0.0001, 'nu': 0.02, 'q': 0.8}
Accuracy: 0.695410, Precision: 0.740445, Recall: 0.601760, F1: 0.663938
Average processing time: 0.008124914222999715
Evaluating parameters: {'gamma': 0.0001, 'nu': 0.05, 'q': 0.4}
Accuracy: 0.657780, Precision: 

In [7]:
import pandas as pd
results = pd.DataFrame(results)
results.to_csv('oneclass.csv', index=False)